# 🎯 Lección 3: XPath y Técnicas Avanzadas de Selección

## 🎯 Objetivos de la Lección

Al finalizar esta lección, serás capaz de:
- Dominar la sintaxis de XPath para selección precisa
- Usar expresiones regulares para extraer patrones específicos
- Implementar técnicas avanzadas de parsing de texto
- Manejar contenido dinámico con Selenium básico
- Combinar múltiples técnicas de selección eficientemente

---

## 🗺️ ¿Qué es XPath?

**XPath** (XML Path Language) es un lenguaje de consulta poderoso para seleccionar nodos en documentos XML y HTML. Es especialmente útil para:

### 🎯 Ventajas de XPath:
- **Precisión**: Selección muy específica de elementos
- **Flexibilidad**: Múltiples criterios de búsqueda
- **Potencia**: Funciones avanzadas y predicados
- **Navegación**: Movimiento libre por el DOM

### 📊 XPath vs CSS Selectors:
| Característica | XPath | CSS Selectors |
|---------------|-------|---------------|
| Sintaxis | `/html/body/div[@class='content']` | `div.content` |
| Texto | `//span[text()='Click me']` | No disponible |
| Navegación | Bidireccional (arriba/abajo) | Solo descendiente |
| Funciones | Muchas funciones built-in | Limitadas |
| Performance | Más lento | Más rápido |


In [ ]:
# 🛠️ Instalación e importación de librerías

!pip install lxml selenium beautifulsoup4 requests fake-useragent webdriver-manager

import requests
from bs4 import BeautifulSoup
from lxml import html, etree
import re
import json
import time
from datetime import datetime
from fake_useragent import UserAgent
from urllib.parse import urljoin, urlparse
import warnings
warnings.filterwarnings('ignore')

# Para Selenium (comentado por ahora)
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager

print("✅ Librerías importadas correctamente")
print(f"📦 Versiones:")
print(f"  - lxml: {html.__version__ if hasattr(html, '__version__') else 'instalado'}")
print(f"  - requests: {requests.__version__}")

## 🔧 Sintaxis Básica de XPath

Comencemos con los fundamentos de XPath:

In [ ]:
# 📚 HTML de ejemplo para practicar XPath

html_xpath_ejemplo = """
<!DOCTYPE html>
<html lang="es">
<head>
    <title>Tienda Online - Electrónicos</title>
</head>
<body>
    <header id="main-header">
        <h1 class="site-title">TechStore</h1>
        <nav class="main-nav">
            <ul>
                <li><a href="/categoria/laptops" data-category="laptops">Laptops</a></li>
                <li><a href="/categoria/smartphones" data-category="phones">Smartphones</a></li>
                <li><a href="/categoria/tablets" data-category="tablets">Tablets</a></li>
            </ul>
        </nav>
    </header>
    
    <main class="content">
        <section class="featured-products">
            <h2>Productos Destacados</h2>
            
            <div class="product-grid">
                <!-- Producto 1 -->
                <article class="product" data-id="P001" data-price="1299.99" data-stock="5">
                    <header class="product-header">
                        <h3 class="product-title">MacBook Pro 14" M3</h3>
                        <span class="product-brand">Apple</span>
                    </header>
                    <div class="product-details">
                        <div class="price-container">
                            <span class="current-price">$1,299.99</span>
                            <span class="original-price">$1,499.99</span>
                            <span class="discount-badge">13% OFF</span>
                        </div>
                        <div class="product-specs">
                            <ul>
                                <li><strong>Procesador:</strong> Apple M3</li>
                                <li><strong>RAM:</strong> 8GB</li>
                                <li><strong>Almacenamiento:</strong> 512GB SSD</li>
                            </ul>
                        </div>
                        <div class="product-rating">
                            <span class="stars">★★★★★</span>
                            <span class="rating-value" data-rating="4.8">4.8</span>
                            <span class="review-count">(124 reviews)</span>
                        </div>
                        <div class="product-badges">
                            <span class="badge new">Nuevo</span>
                            <span class="badge free-shipping">Envío Gratis</span>
                        </div>
                    </div>
                    <footer class="product-footer">
                        <button class="btn-add-cart" data-product-id="P001">Agregar al Carrito</button>
                        <a href="/producto/macbook-pro-14-m3" class="btn-details">Ver Detalles</a>
                    </footer>
                </article>
                
                <!-- Producto 2 -->
                <article class="product" data-id="P002" data-price="899.99" data-stock="12">
                    <header class="product-header">
                        <h3 class="product-title">iPhone 15 Pro</h3>
                        <span class="product-brand">Apple</span>
                    </header>
                    <div class="product-details">
                        <div class="price-container">
                            <span class="current-price">$899.99</span>
                            <span class="original-price">$999.99</span>
                            <span class="discount-badge">10% OFF</span>
                        </div>
                        <div class="product-specs">
                            <ul>
                                <li><strong>Procesador:</strong> A17 Pro</li>
                                <li><strong>Pantalla:</strong> 6.1" Super Retina XDR</li>
                                <li><strong>Almacenamiento:</strong> 128GB</li>
                            </ul>
                        </div>
                        <div class="product-rating">
                            <span class="stars">★★★★☆</span>
                            <span class="rating-value" data-rating="4.5">4.5</span>
                            <span class="review-count">(89 reviews)</span>
                        </div>
                        <div class="product-badges">
                            <span class="badge bestseller">Best Seller</span>
                        </div>
                    </div>
                    <footer class="product-footer">
                        <button class="btn-add-cart" data-product-id="P002">Agregar al Carrito</button>
                        <a href="/producto/iphone-15-pro" class="btn-details">Ver Detalles</a>
                    </footer>
                </article>
                
                <!-- Producto 3 -->
                <article class="product" data-id="P003" data-price="249.99" data-stock="0">
                    <header class="product-header">
                        <h3 class="product-title">Samsung Galaxy Tab S9</h3>
                        <span class="product-brand">Samsung</span>
                    </header>
                    <div class="product-details">
                        <div class="price-container">
                            <span class="current-price">$249.99</span>
                            <span class="out-of-stock">Sin Stock</span>
                        </div>
                        <div class="product-specs">
                            <ul>
                                <li><strong>Procesador:</strong> Snapdragon 8 Gen 2</li>
                                <li><strong>Pantalla:</strong> 11" Dynamic AMOLED</li>
                                <li><strong>RAM:</strong> 8GB</li>
                            </ul>
                        </div>
                        <div class="product-rating">
                            <span class="stars">★★★★★</span>
                            <span class="rating-value" data-rating="4.7">4.7</span>
                            <span class="review-count">(56 reviews)</span>
                        </div>
                        <div class="product-badges">
                            <span class="badge premium">Premium</span>
                        </div>
                    </div>
                    <footer class="product-footer">
                        <button class="btn-notify" data-product-id="P003" disabled>Notificar Disponibilidad</button>
                        <a href="/producto/samsung-galaxy-tab-s9" class="btn-details">Ver Detalles</a>
                    </footer>
                </article>
            </div>
        </section>
        
        <!-- Sidebar con filtros -->
        <aside class="sidebar">
            <section class="filter-section">
                <h3>Filtrar por Precio</h3>
                <div class="price-ranges">
                    <label><input type="checkbox" data-min="0" data-max="500"> Menos de $500</label>
                    <label><input type="checkbox" data-min="500" data-max="1000"> $500 - $1,000</label>
                    <label><input type="checkbox" data-min="1000" data-max="2000"> $1,000 - $2,000</label>
                    <label><input type="checkbox" data-min="2000" data-max="99999"> Más de $2,000</label>
                </div>
            </section>
            
            <section class="brand-section">
                <h3>Marcas Disponibles</h3>
                <ul class="brand-list">
                    <li data-brand="apple">Apple <span class="count">(2)</span></li>
                    <li data-brand="samsung">Samsung <span class="count">(1)</span></li>
                    <li data-brand="dell">Dell <span class="count">(0)</span></li>
                    <li data-brand="hp">HP <span class="count">(0)</span></li>
                </ul>
            </section>
        </aside>
    </main>
    
    <!-- Footer con información adicional -->
    <footer class="site-footer">
        <div class="footer-info">
            <p>© 2024 TechStore. Todos los derechos reservados.</p>
            <p>Contacto: <span class="phone">+1-555-0123</span> | Email: <span class="email">info@techstore.com</span></p>
        </div>
        <div class="footer-stats" data-total-products="150" data-total-brands="12">
            <span>150 productos disponibles</span>
        </div>
    </footer>
</body>
</html>
"""

# Parsear con lxml para XPath
tree = html.fromstring(html_xpath_ejemplo)

# También parsear con BeautifulSoup para comparaciones
soup = BeautifulSoup(html_xpath_ejemplo, 'html.parser')

print("✅ HTML parseado con lxml y BeautifulSoup")
print(f"📊 Elementos totales (lxml): {len(tree.xpath('//*'))}")
print(f"📊 Elementos totales (BeautifulSoup): {len(soup.find_all())}")

## 🔍 Selectores XPath Básicos

### 📝 Sintaxis fundamental:

In [ ]:
# 🎯 XPath básico - Selectores fundamentales

def demo_xpath_basico(tree):
    """Demostración de selectores XPath básicos"""
    
    print("🎯 SELECTORES XPATH BÁSICOS")
    print("="*30)
    
    # 1. Selección absoluta vs relativa
    print("\n📍 1. Rutas absolutas vs relativas:")
    
    # Ruta absoluta (desde la raíz)
    titulo_absoluto = tree.xpath('/html/head/title')
    print(f"  📋 Título (ruta absoluta): {titulo_absoluto[0].text if titulo_absoluto else 'No encontrado'}")
    
    # Ruta relativa (buscar en cualquier lugar)
    titulos_relativos = tree.xpath('//title')
    print(f"  📋 Títulos (ruta relativa): {[t.text for t in titulos_relativos]}")
    
    # 2. Selección por nombre de elemento
    print("\n🏷️ 2. Selección por elemento:")
    
    # Todos los h3
    h3_elementos = tree.xpath('//h3')
    print(f"  📝 Elementos H3 encontrados: {len(h3_elementos)}")
    for i, h3 in enumerate(h3_elementos, 1):
        print(f"    {i}. {h3.text}")
    
    # 3. Selección por atributos
    print("\n🔧 3. Selección por atributos:")
    
    # Elementos con clase específica
    productos = tree.xpath('//article[@class="product"]')
    print(f"  🛍️ Productos encontrados: {len(productos)}")
    
    # Elementos con data-id específico
    producto_p001 = tree.xpath('//article[@data-id="P001"]')
    if producto_p001:
        titulo = producto_p001[0].xpath('.//h3[@class="product-title"]')[0].text
        print(f"  🎯 Producto P001: {titulo}")
    
    # 4. Selección por índice
    print("\n🔢 4. Selección por índice:")
    
    # Primer producto
    primer_producto = tree.xpath('//article[@class="product"][1]')
    if primer_producto:
        titulo = primer_producto[0].xpath('.//h3')[0].text
        print(f"  1️⃣ Primer producto: {titulo}")
    
    # Último producto
    ultimo_producto = tree.xpath('//article[@class="product"][last()]')
    if ultimo_producto:
        titulo = ultimo_producto[0].xpath('.//h3')[0].text
        print(f"  🔚 Último producto: {titulo}")
    
    # 5. Selección por contenido de texto
    print("\n📝 5. Selección por contenido de texto:")
    
    # Elemento que contiene texto específico
    apple_brand = tree.xpath('//span[text()="Apple"]')
    print(f"  🍎 Productos Apple encontrados: {len(apple_brand)}")
    
    # Texto que contiene una palabra
    productos_con_pro = tree.xpath('//h3[contains(text(), "Pro")]')
    print(f"  ⭐ Productos 'Pro': {[p.text for p in productos_con_pro]}")

# Ejecutar demo
demo_xpath_basico(tree)

## 🎭 XPath Avanzado con Predicados

Los predicados nos permiten crear condiciones más complejas:

In [ ]:
# 🎭 XPath avanzado con predicados y funciones

def demo_xpath_avanzado(tree):
    """Demostración de XPath avanzado con predicados"""
    
    print("🎭 XPATH AVANZADO CON PREDICADOS")
    print("="*35)
    
    # 1. Predicados con operadores de comparación
    print("\n🔢 1. Predicados numéricos:")
    
    # Productos con precio mayor a 500
    productos_caros = tree.xpath('//article[@data-price > 500]')
    print(f"  💰 Productos > $500: {len(productos_caros)}")
    for producto in productos_caros:
        titulo = producto.xpath('.//h3')[0].text
        precio = producto.get('data-price')
        print(f"    • {titulo}: ${precio}")
    
    # Productos con stock disponible
    productos_disponibles = tree.xpath('//article[@data-stock > 0]')
    print(f"\n  📦 Productos en stock: {len(productos_disponibles)}")
    for producto in productos_disponibles:
        titulo = producto.xpath('.//h3')[0].text
        stock = producto.get('data-stock')
        print(f"    • {titulo}: {stock} unidades")
    
    # 2. Múltiples condiciones con 'and' y 'or'
    print("\n🔗 2. Múltiples condiciones:")
    
    # Productos Apple con stock
    apple_en_stock = tree.xpath('//article[@data-stock > 0 and .//span[text()="Apple"]]')
    print(f"  🍎 Productos Apple en stock: {len(apple_en_stock)}")
    for producto in apple_en_stock:
        titulo = producto.xpath('.//h3')[0].text
        print(f"    • {titulo}")
    
    # Productos caros O sin stock
    caros_o_sin_stock = tree.xpath('//article[@data-price > 800 or @data-stock = 0]')
    print(f"\n  💸 Productos caros o sin stock: {len(caros_o_sin_stock)}")
    for producto in caros_o_sin_stock:
        titulo = producto.xpath('.//h3')[0].text
        precio = producto.get('data-price')
        stock = producto.get('data-stock')
        print(f"    • {titulo}: ${precio} (Stock: {stock})")
    
    # 3. Funciones de texto avanzadas
    print("\n📝 3. Funciones de texto:")
    
    # starts-with(): elementos que empiezan con un texto
    productos_mac = tree.xpath('//h3[starts-with(text(), "Mac")]')
    print(f"  🖥️ Productos que empiezan con 'Mac': {[p.text for p in productos_mac]}")
    
    # contains(): elementos que contienen texto
    productos_con_numeros = tree.xpath('//h3[contains(text(), "15") or contains(text(), "14")]')
    print(f"  🔢 Productos con números (14, 15): {[p.text for p in productos_con_numeros]}")
    
    # normalize-space(): ignora espacios extra
    badges = tree.xpath('//span[@class="badge" and normalize-space(text())="Nuevo"]')
    print(f"  🏷️ Badges 'Nuevo': {len(badges)}")
    
    # 4. Navegación con ejes
    print("\n🧭 4. Navegación con ejes:")
    
    # following-sibling: hermanos siguientes
    primer_producto = tree.xpath('//article[@data-id="P001"]')[0]
    hermanos_siguientes = primer_producto.xpath('./following-sibling::article')
    print(f"  ➡️ Productos después de P001: {len(hermanos_siguientes)}")
    
    # parent: elemento padre
    precio_actual = tree.xpath('//span[@class="current-price"][1]')[0]
    contenedor_precio = precio_actual.xpath('./parent::div')[0]
    print(f"  👆 Contenedor del precio: <{contenedor_precio.tag}> clase '{contenedor_precio.get('class')}'")
    
    # ancestor: antecesores
    boton_carrito = tree.xpath('//button[@class="btn-add-cart"][1]')[0]
    producto_ancestro = boton_carrito.xpath('./ancestor::article')[0]
    titulo_ancestro = producto_ancestro.xpath('.//h3')[0].text
    print(f"  🏛️ Producto ancestro del botón: {titulo_ancestro}")
    
    # 5. Funciones de posición y conteo
    print("\n📊 5. Funciones de posición:")
    
    # position(): posición del elemento
    segundo_producto = tree.xpath('//article[@class="product"][position()=2]')
    if segundo_producto:
        titulo = segundo_producto[0].xpath('.//h3')[0].text
        print(f"  2️⃣ Segundo producto: {titulo}")
    
    # count(): contar elementos
    total_specs = tree.xpath('count(//div[@class="product-specs"]//li)')
    print(f"  📋 Total de especificaciones: {int(total_specs)}")
    
    # 6. Operadores lógicos complejos
    print("\n🧠 6. Lógica compleja:")
    
    # not(): negación
    productos_no_apple = tree.xpath('//article[not(.//span[text()="Apple"])]')
    print(f"  🚫 Productos NO Apple: {len(productos_no_apple)}")
    for producto in productos_no_apple:
        titulo = producto.xpath('.//h3')[0].text
        marca = producto.xpath('.//span[@class="product-brand"]')[0].text
        print(f"    • {titulo} ({marca})")

# Ejecutar demo
demo_xpath_avanzado(tree)

## 🔤 Expresiones Regulares para Extracción de Patrones

Las expresiones regulares (regex) son perfectas para extraer patrones específicos del texto:

In [ ]:
# 🔤 Expresiones regulares para extracción de patrones

import re

def demo_regex_extraccion(tree):
    """Demostración de extracción con expresiones regulares"""
    
    print("🔤 EXPRESIONES REGULARES PARA EXTRACCIÓN")
    print("="*45)
    
    # Obtener todo el texto del HTML para trabajar
    texto_completo = etree.tostring(tree, method='text', encoding='unicode')
    
    # 1. Extracción de precios
    print("\n💰 1. Extracción de precios:")
    
    # Patrón para precios en formato $X,XXX.XX
    patron_precios = r'\$([0-9]{1,3}(?:,[0-9]{3})*(?:\.[0-9]{2})?)'
    precios_encontrados = re.findall(patron_precios, texto_completo)
    
    print(f"  💵 Precios encontrados: {len(precios_encontrados)}")
    for i, precio in enumerate(precios_encontrados, 1):
        print(f"    {i}. ${precio}")
    
    # Convertir a números para análisis
    precios_numericos = [float(p.replace(',', '')) for p in precios_encontrados]
    if precios_numericos:
        print(f"  📊 Precio promedio: ${sum(precios_numericos)/len(precios_numericos):.2f}")
        print(f"  📈 Precio máximo: ${max(precios_numericos):.2f}")
        print(f"  📉 Precio mínimo: ${min(precios_numericos):.2f}")
    
    # 2. Extracción de especificaciones técnicas
    print("\n🔧 2. Extracción de especificaciones:")
    
    # Extraer capacidades de almacenamiento
    patron_storage = r'([0-9]+(?:GB|TB))\s*(?:SSD|HDD|storage|almacenamiento)?'
    almacenamientos = re.findall(patron_storage, texto_completo, re.IGNORECASE)
    print(f"  💾 Capacidades encontradas: {set(almacenamientos)}")
    
    # Extraer información de RAM
    patron_ram = r'([0-9]+GB)\s*(?:RAM|de memoria)'
    ram_encontrada = re.findall(patron_ram, texto_completo, re.IGNORECASE)
    print(f"  🧠 RAM encontrada: {set(ram_encontrada)}")
    
    # Extraer tamaños de pantalla
    patron_pantalla = r'([0-9]+(?:\.[0-9]+)?")\s*(?:[A-Za-z\s]*(?:pantalla|screen|display))'
    pantallas = re.findall(patron_pantalla, texto_completo, re.IGNORECASE)
    print(f"  📺 Tamaños de pantalla: {set(pantallas)}")
    
    # 3. Extracción de descuentos y ofertas
    print("\n🏷️ 3. Extracción de descuentos:")
    
    # Patrón para descuentos
    patron_descuentos = r'([0-9]+)%\s*(?:OFF|descuento|desc)'
    descuentos = re.findall(patron_descuentos, texto_completo, re.IGNORECASE)
    print(f"  🔥 Descuentos encontrados: {[d + '%' for d in descuentos]}")
    
    # Calcular descuento promedio
    if descuentos:
        descuentos_numericos = [int(d) for d in descuentos]
        print(f"  📊 Descuento promedio: {sum(descuentos_numericos)/len(descuentos_numericos):.1f}%")
    
    # 4. Extracción de ratings y reviews
    print("\n⭐ 4. Extracción de ratings:")
    
    # Ratings en formato X.X
    patron_ratings = r'([0-5]\.[0-9])\s*(?:stars?|estrellas?)'
    ratings = re.findall(patron_ratings, texto_completo, re.IGNORECASE)
    print(f"  ⭐ Ratings encontrados: {ratings}")
    
    # Número de reviews
    patron_reviews = r'\((\d+)\s*reviews?\)'
    reviews = re.findall(patron_reviews, texto_completo, re.IGNORECASE)
    print(f"  💬 Cantidad de reviews: {reviews}")
    
    # 5. Extracción de información de contacto
    print("\n📞 5. Información de contacto:")
    
    # Teléfonos
    patron_telefono = r'\+?[0-9]{1,3}[-\s]?\(?[0-9]{3}\)?[-\s]?[0-9]{3,4}[-\s]?[0-9]{3,4}'
    telefonos = re.findall(patron_telefono, texto_completo)
    print(f"  📞 Teléfonos: {telefonos}")
    
    # Emails
    patron_email = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(patron_email, texto_completo)
    print(f"  📧 Emails: {emails}")
    
    # 6. Extracción avanzada con grupos nombrados
    print("\n🎯 6. Extracción con grupos nombrados:")
    
    # Patrón complejo para productos
    patron_producto = r'(?P<marca>Apple|Samsung|Dell|HP)\s+(?P<modelo>[A-Za-z0-9\s]+?)\s*(?P<version>[0-9]+["]*)?'
    
    # Buscar en títulos específicos
    titulos_productos = tree.xpath('//h3[@class="product-title"]/text()')
    
    print(f"  🏷️ Análisis de productos:")
    for titulo in titulos_productos:
        match = re.search(patron_producto, titulo, re.IGNORECASE)
        if match:
            grupos = match.groupdict()
            print(f"    📱 Título: {titulo}")
            print(f"      • Marca: {grupos.get('marca', 'N/A')}")
            print(f"      • Modelo: {grupos.get('modelo', 'N/A').strip()}")
            print(f"      • Versión: {grupos.get('version', 'N/A')}")

# Ejecutar demo
demo_regex_extraccion(tree)

## 🔗 Combinando XPath, CSS Selectors y Regex

La verdadera potencia viene de combinar todas las técnicas:

In [ ]:
# 🔗 Combinando todas las técnicas de selección

def extractor_completo_productos(tree, soup):
    """Extractor completo que combina XPath, CSS y Regex"""
    
    print("🔗 EXTRACTOR COMPLETO - COMBINANDO TÉCNICAS")
    print("="*50)
    
    productos_extraidos = []
    
    # 1. Usar XPath para obtener todos los productos
    productos_xpath = tree.xpath('//article[@class="product"]')
    print(f"🎯 Productos encontrados con XPath: {len(productos_xpath)}")
    
    for i, producto_elem in enumerate(productos_xpath, 1):
        print(f"\n📦 Procesando producto {i}:")
        
        # Información básica con XPath
        producto_data = {
            'id': producto_elem.get('data-id'),
            'precio_raw': producto_elem.get('data-price'),
            'stock': int(producto_elem.get('data-stock', 0))
        }
        
        # Título y marca con XPath
        titulo_elem = producto_elem.xpath('.//h3[@class="product-title"]')
        marca_elem = producto_elem.xpath('.//span[@class="product-brand"]')
        
        if titulo_elem:
            producto_data['titulo'] = titulo_elem[0].text.strip()
        if marca_elem:
            producto_data['marca'] = marca_elem[0].text.strip()
        
        print(f"  🏷️ {producto_data.get('titulo', 'Sin título')} ({producto_data.get('marca', 'Sin marca')})")
        
        # 2. Usar CSS selectors con BeautifulSoup para precios
        producto_id = producto_data['id']
        producto_soup = soup.find('article', {'data-id': producto_id})
        
        if producto_soup:
            # Precio actual
            precio_actual = producto_soup.select_one('.current-price')
            precio_original = producto_soup.select_one('.original-price')
            discount_badge = producto_soup.select_one('.discount-badge')
            
            if precio_actual:
                producto_data['precio_display'] = precio_actual.text.strip()
            if precio_original:
                producto_data['precio_original'] = precio_original.text.strip()
            if discount_badge:
                producto_data['descuento'] = discount_badge.text.strip()
        
        # 3. Usar regex para extraer y limpiar datos
        if 'precio_display' in producto_data:
            # Extraer precio numérico con regex
            precio_match = re.search(r'\$([0-9,]+\.?[0-9]*)', producto_data['precio_display'])
            if precio_match:
                precio_numerico = float(precio_match.group(1).replace(',', ''))
                producto_data['precio_numerico'] = precio_numerico
        
        if 'descuento' in producto_data:
            # Extraer porcentaje de descuento
            descuento_match = re.search(r'([0-9]+)%', producto_data['descuento'])
            if descuento_match:
                producto_data['descuento_porcentaje'] = int(descuento_match.group(1))
        
        # 4. Combinar XPath y regex para especificaciones
        specs_elementos = producto_elem.xpath('.//div[@class="product-specs"]//li')
        especificaciones = {}
        
        for spec in specs_elementos:
            texto_spec = spec.text or etree.tostring(spec, method='text', encoding='unicode')
            
            # Usar regex para parsear especificaciones
            if 'Procesador:' in texto_spec:
                proc_match = re.search(r'Procesador:\s*(.+)', texto_spec)
                if proc_match:
                    especificaciones['procesador'] = proc_match.group(1).strip()
            
            elif 'RAM:' in texto_spec:
                ram_match = re.search(r'RAM:\s*([0-9]+GB)', texto_spec)
                if ram_match:
                    especificaciones['ram'] = ram_match.group(1)
            
            elif 'Almacenamiento:' in texto_spec:
                storage_match = re.search(r'Almacenamiento:\s*([0-9]+GB\s*SSD)', texto_spec)
                if storage_match:
                    especificaciones['almacenamiento'] = storage_match.group(1)
            
            elif 'Pantalla:' in texto_spec:
                pantalla_match = re.search(r'Pantalla:\s*(.+)', texto_spec)
                if pantalla_match:
                    especificaciones['pantalla'] = pantalla_match.group(1).strip()
        
        producto_data['especificaciones'] = especificaciones
        
        # 5. Rating y reviews con XPath + regex
        rating_elem = producto_elem.xpath('.//span[@class="rating-value"]')
        review_elem = producto_elem.xpath('.//span[@class="review-count"]')
        
        if rating_elem:
            producto_data['rating'] = float(rating_elem[0].text.strip())
        
        if review_elem:
            review_text = review_elem[0].text
            review_match = re.search(r'\((\d+)', review_text)
            if review_match:
                producto_data['num_reviews'] = int(review_match.group(1))
        
        # 6. Badges y características especiales
        badges_elementos = producto_elem.xpath('.//span[@class="badge"]')
        badges = [badge.text.strip() for badge in badges_elementos if badge.text]
        producto_data['badges'] = badges
        
        # Determinar disponibilidad
        producto_data['disponible'] = producto_data['stock'] > 0
        producto_data['estado'] = 'Disponible' if producto_data['disponible'] else 'Sin Stock'
        
        # Agregar producto a la lista
        productos_extraidos.append(producto_data)
        
        # Mostrar resumen del producto
        print(f"    💰 Precio: ${producto_data.get('precio_numerico', 'N/A')}")
        print(f"    📦 Stock: {producto_data['stock']} ({producto_data['estado']})")
        print(f"    ⭐ Rating: {producto_data.get('rating', 'N/A')} ({producto_data.get('num_reviews', 0)} reviews)")
        print(f"    🔧 Specs: {len(especificaciones)} especificaciones")
        print(f"    🏷️ Badges: {', '.join(badges) if badges else 'Ninguno'}")
    
    return productos_extraidos

# Ejecutar extractor completo
productos_completos = extractor_completo_productos(tree, soup)

print(f"\n✅ EXTRACCIÓN COMPLETADA")
print(f"📊 Total de productos procesados: {len(productos_completos)}")
print(f"📈 Productos disponibles: {sum(1 for p in productos_completos if p['disponible'])}")
print(f"📉 Productos sin stock: {sum(1 for p in productos_completos if not p['disponible'])}")

# Análisis rápido
precios_disponibles = [p['precio_numerico'] for p in productos_completos if p.get('precio_numerico') and p['disponible']]
if precios_disponibles:
    print(f"💰 Rango de precios disponibles: ${min(precios_disponibles):.2f} - ${max(precios_disponibles):.2f}")
    print(f"📊 Precio promedio: ${sum(precios_disponibles)/len(precios_disponibles):.2f}")

## 🤖 Introducción a Selenium para Contenido Dinámico

Cuando el contenido se carga dinámicamente con JavaScript, necesitamos Selenium:

In [ ]:
# 🤖 Simulación de Selenium (sin navegador real)
# Nota: Este es un ejemplo educativo. Para usar Selenium real, necesitas instalar un driver del navegador

def simular_contenido_dinamico():
    """Simulación de cómo funcionaría Selenium con contenido dinámico"""
    
    print("🤖 SIMULACIÓN: CONTENIDO DINÁMICO CON SELENIUM")
    print("="*50)
    
    # HTML que simula contenido que se cargaría dinámicamente
    html_dinamico_inicial = """
    <div id="productos-container">
        <div class="loading">Cargando productos...</div>
        <button id="load-more">Cargar más productos</button>
    </div>
    <script>
        // JavaScript que cargaría contenido dinámicamente
        setTimeout(function() {
            loadProducts();
        }, 2000);
    </script>
    """
    
    # HTML después de que JavaScript cargue el contenido
    html_dinamico_final = """
    <div id="productos-container">
        <div class="producto-dinamico" data-id="D001">
            <h3>Producto Dinámico 1</h3>
            <p class="precio-dinamico">$599.99</p>
        </div>
        <div class="producto-dinamico" data-id="D002">
            <h3>Producto Dinámico 2</h3>
            <p class="precio-dinamico">$799.99</p>
        </div>
        <button id="load-more">Cargar más productos</button>
    </div>
    """
    
    print("🔄 Estado inicial (antes de JavaScript):")
    soup_inicial = BeautifulSoup(html_dinamico_inicial, 'html.parser')
    productos_iniciales = soup_inicial.find_all('div', class_='producto-dinamico')
    print(f"  📦 Productos encontrados: {len(productos_iniciales)}")
    loading_div = soup_inicial.find('div', class_='loading')
    print(f"  ⏳ Loading presente: {'Sí' if loading_div else 'No'}")
    
    print("\n⏱️ Simulando espera de JavaScript (2 segundos)...")
    time.sleep(2)
    
    print("\n✅ Estado final (después de JavaScript):")
    soup_final = BeautifulSoup(html_dinamico_final, 'html.parser')
    productos_finales = soup_final.find_all('div', class_='producto-dinamico')
    print(f"  📦 Productos encontrados: {len(productos_finales)}")
    
    for producto in productos_finales:
        titulo = producto.find('h3').text
        precio = producto.find('p', class_='precio-dinamico').text
        print(f"    • {titulo}: {precio}")
    
    return soup_final

# Ejecutar simulación
resultado_dinamico = simular_contenido_dinamico()

print("\n📝 CÓDIGO SELENIUM REAL (comentado):")
print("""
# Código que usarías con Selenium real:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configurar el driver
driver = webdriver.Chrome()

try:
    # Navegar a la página
    driver.get("https://ejemplo.com/productos")
    
    # Esperar a que aparezcan los productos dinámicos
    wait = WebDriverWait(driver, 10)
    productos = wait.until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "producto-dinamico"))
    )
    
    # Hacer clic en "Cargar más" si existe
    load_more_btn = driver.find_element(By.ID, "load-more")
    if load_more_btn.is_displayed():
        load_more_btn.click()
        
        # Esperar a que carguen más productos
        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "producto-dinamico"))
        )
    
    # Obtener el HTML final después de que JavaScript termine
    html_final = driver.page_source
    
    # Parsear con BeautifulSoup
    soup = BeautifulSoup(html_final, 'html.parser')
    
finally:
    driver.quit()
""")

## 🧪 Ejercicios Prácticos

### 📝 Ejercicio 1: Master de XPath

In [ ]:
# 📝 Ejercicio 1: Convertirse en un maestro de XPath

print("🧪 EJERCICIO 1: MAESTRO DE XPATH")
print("="*35)

def ejercicio_xpath_maestro(tree):
    """Ejercicio para dominar XPath con desafíos progresivos"""
    
    resultados = {}
    
    print("\n🎯 DESAFÍO 1: Selectores Básicos")
    print("Encuentra:")
    
    # Desafío 1.1: Todos los productos con descuento
    print("  1.1. Todos los productos que tienen descuento")
    productos_con_descuento = tree.xpath('//article[.//span[@class="discount-badge"]]')
    resultados['productos_con_descuento'] = len(productos_con_descuento)
    print(f"       ✅ Encontrados: {len(productos_con_descuento)}")
    
    # Desafío 1.2: El producto más caro disponible
    print("  1.2. El producto más caro que esté disponible (stock > 0)")
    productos_disponibles = tree.xpath('//article[@data-stock > 0]')
    if productos_disponibles:
        precios = [float(p.get('data-price', 0)) for p in productos_disponibles]
        precio_maximo = max(precios)
        producto_caro = tree.xpath(f'//article[@data-stock > 0 and @data-price="{precio_maximo}"]')[0]
        titulo_caro = producto_caro.xpath('.//h3')[0].text
        resultados['producto_mas_caro'] = {'titulo': titulo_caro, 'precio': precio_maximo}
        print(f"       ✅ {titulo_caro}: ${precio_maximo}")
    
    print("\n🎯 DESAFÍO 2: Predicados Avanzados")
    
    # Desafío 2.1: Productos Apple con rating > 4.5
    print("  2.1. Productos de Apple con rating mayor a 4.5")
    apple_high_rating = tree.xpath('//article[.//span[text()="Apple"] and .//span[@data-rating > 4.5]]')
    resultados['apple_high_rating'] = len(apple_high_rating)
    print(f"       ✅ Encontrados: {len(apple_high_rating)}")
    for producto in apple_high_rating:
        titulo = producto.xpath('.//h3')[0].text
        rating = producto.xpath('.//span[@class="rating-value"]')[0].text
        print(f"         • {titulo} (Rating: {rating})")
    
    # Desafío 2.2: Productos con especificaciones específicas
    print("  2.2. Productos que mencionen 'SSD' en sus especificaciones")
    productos_ssd = tree.xpath('//article[.//li[contains(text(), "SSD")]]')
    resultados['productos_ssd'] = len(productos_ssd)
    print(f"       ✅ Encontrados: {len(productos_ssd)}")
    
    print("\n🎯 DESAFÍO 3: Navegación Compleja")
    
    # Desafío 3.1: Encontrar hermanos de productos sin stock
    print("  3.1. Productos que son hermanos de productos sin stock")
    productos_sin_stock = tree.xpath('//article[@data-stock = 0]')
    hermanos_sin_stock = []
    for producto in productos_sin_stock:
        hermanos = producto.xpath('./following-sibling::article | ./preceding-sibling::article')
        hermanos_sin_stock.extend(hermanos)
    
    hermanos_unicos = list(set(hermanos_sin_stock))  # Eliminar duplicados
    resultados['hermanos_sin_stock'] = len(hermanos_unicos)
    print(f"       ✅ Hermanos encontrados: {len(hermanos_unicos)}")
    
    # Desafío 3.2: Contenedores padre de precios con descuento
    print("  3.2. Contenedores padre de elementos con descuento")
    discount_badges = tree.xpath('//span[@class="discount-badge"]')
    contenedores_descuento = []
    for badge in discount_badges:
        contenedor = badge.xpath('./ancestor::div[@class="price-container"]')
        contenedores_descuento.extend(contenedor)
    
    resultados['contenedores_descuento'] = len(contenedores_descuento)
    print(f"       ✅ Contenedores encontrados: {len(contenedores_descuento)}")
    
    print("\n🎯 DESAFÍO 4: Funciones Avanzadas")
    
    # Desafío 4.1: Contar total de especificaciones
    print("  4.1. Total de elementos <li> en especificaciones")
    total_specs = tree.xpath('count(//div[@class="product-specs"]//li)')
    resultados['total_especificaciones'] = int(total_specs)
    print(f"       ✅ Total: {int(total_specs)} especificaciones")
    
    # Desafío 4.2: Productos cuyo título tiene más de 15 caracteres
    print("  4.2. Productos con títulos largos (> 15 caracteres)")
    titulos_largos = tree.xpath('//h3[@class="product-title"][string-length(text()) > 15]')
    resultados['titulos_largos'] = len(titulos_largos)
    print(f"       ✅ Encontrados: {len(titulos_largos)}")
    for titulo in titulos_largos:
        print(f"         • {titulo.text} ({len(titulo.text)} caracteres)")
    
    print("\n🎯 DESAFÍO 5: Lógica Compleja")
    
    # Desafío 5.1: Productos que NO son Apple Y tienen stock
    print("  5.1. Productos que NO son Apple pero SÍ tienen stock")
    no_apple_con_stock = tree.xpath('//article[@data-stock > 0 and not(.//span[text()="Apple"])]')
    resultados['no_apple_con_stock'] = len(no_apple_con_stock)
    print(f"       ✅ Encontrados: {len(no_apple_con_stock)}")
    for producto in no_apple_con_stock:
        titulo = producto.xpath('.//h3')[0].text
        marca = producto.xpath('.//span[@class="product-brand"]')[0].text
        print(f"         • {titulo} ({marca})")
    
    # Desafío 5.2: El segundo enlace de navegación
    print("  5.2. El segundo enlace del menú de navegación")
    segundo_enlace = tree.xpath('//nav[@class="main-nav"]//li[2]/a')
    if segundo_enlace:
        texto_enlace = segundo_enlace[0].text
        href_enlace = segundo_enlace[0].get('href')
        resultados['segundo_enlace'] = {'texto': texto_enlace, 'href': href_enlace}
        print(f"       ✅ {texto_enlace} -> {href_enlace}")
    
    return resultados

# Ejecutar ejercicio
resultados_ejercicio = ejercicio_xpath_maestro(tree)

print("\n🏆 RESUMEN DEL EJERCICIO:")
print("="*30)
for clave, valor in resultados_ejercicio.items():
    print(f"📊 {clave}: {valor}")

# Puntuación del ejercicio (simulada)
puntos_totales = sum([1 for v in resultados_ejercicio.values() if v])
print(f"\n🎯 Puntuación: {puntos_totales}/10 desafíos completados")
if puntos_totales >= 8:
    print("🏆 ¡Excelente! Eres un maestro de XPath")
elif puntos_totales >= 6:
    print("👍 ¡Bien! Tienes buen dominio de XPath")
else:
    print("📚 Sigue practicando para mejorar tu XPath")

### 📝 Ejercicio 2: Extractor con Expresiones Regulares

In [ ]:
# 📝 Ejercicio 2: Crear un extractor avanzado con regex

print("🧪 EJERCICIO 2: EXTRACTOR REGEX AVANZADO")
print("="*45)

def ejercicio_regex_avanzado():
    """Crear un sistema de extracción basado en regex para diferentes tipos de datos"""
    
    # Texto de ejemplo con información mezclada
    texto_mixto = """
    Información de Contacto:
    - Teléfonos: +1-555-123-4567, (555) 987-6543, 555.456.7890
    - Emails: soporte@techstore.com, ventas@techstore.es, info@tech-store.net
    - Direcciones: 123 Main St, New York NY 10001, 456 Oak Ave, Los Angeles CA 90210
    
    Productos y Precios:
    - MacBook Pro 16" M3 por $2,499.99 (antes $2,799.99) - 15% de descuento
    - iPhone 15 Pro Max 256GB a $1,199.00 - Envío gratis
    - Samsung Galaxy S24 Ultra desde $899.50 hasta $1,299.99
    - iPad Air 5ta generación: $599 (stock: 25 unidades)
    
    Especificaciones Técnicas:
    - Procesador: Apple M3 Pro de 12 núcleos
    - Memoria RAM: 32GB LPDDR5
    - Almacenamiento: 1TB SSD NVMe
    - Pantalla: 16.2 pulgadas Liquid Retina XDR
    - Batería: 100Wh Li-Po
    - Peso: 2.15kg
    
    Fechas y Horarios:
    - Lanzamiento: 2024-01-15
    - Disponible desde: 15/01/2024
    - Evento de presentación: January 15th, 2024 at 10:00 AM PST
    - Garantía válida hasta: 2025-01-15
    
    URLs y Enlaces:
    - Sitio web: https://www.techstore.com/productos/macbook-pro
    - Soporte: https://support.techstore.com
    - Manual: ftp://files.techstore.com/manuals/macbook_pro_manual.pdf
    - API: http://api.techstore.com/v2/products
    
    Códigos y Referencias:
    - SKU: MBP-16-M3-1TB-32GB
    - Código de barras: 194252847695
    - Serial: C02DQ2XQMD6T
    - Modelo: A2780
    """
    
    print("📝 Texto de ejemplo cargado (fragmento):")
    print(texto_mixto[:200] + "...")
    
    print("\n🔍 EXTRAYENDO INFORMACIÓN CON REGEX:")
    print("="*45)
    
    extracciones = {}
    
    # 1. Información de contacto
    print("\n📞 1. Información de Contacto:")
    
    # Teléfonos (múltiples formatos)
    patron_telefono = r'(?:\+?1[-\s]?)?\(?([0-9]{3})\)?[-\s\.]?([0-9]{3})[-\s\.]?([0-9]{4})'
    telefonos = re.findall(patron_telefono, texto_mixto)
    telefonos_formateados = [f"({t[0]}) {t[1]}-{t[2]}" for t in telefonos]
    extracciones['telefonos'] = telefonos_formateados
    print(f"  📱 Teléfonos: {telefonos_formateados}")
    
    # Emails
    patron_email = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(patron_email, texto_mixto)
    extracciones['emails'] = emails
    print(f"  📧 Emails: {emails}")
    
    # 2. Productos y precios
    print("\n💰 2. Productos y Precios:")
    
    # Precios en formato $X,XXX.XX
    patron_precios = r'\$([0-9]{1,3}(?:,[0-9]{3})*(?:\.[0-9]{2})?|[0-9]+(?:\.[0-9]{2})?)'
    precios = re.findall(patron_precios, texto_mixto)
    precios_numericos = [float(p.replace(',', '')) for p in precios]
    extracciones['precios'] = precios_numericos
    print(f"  💵 Precios encontrados: ${', $'.join(precios)}")
    print(f"  📊 Rango: ${min(precios_numericos):.2f} - ${max(precios_numericos):.2f}")
    
    # Descuentos
    patron_descuentos = r'([0-9]+)%\s*de\s*descuento'
    descuentos = re.findall(patron_descuentos, texto_mixto, re.IGNORECASE)
    extracciones['descuentos'] = [int(d) for d in descuentos]
    print(f"  🏷️ Descuentos: {[d + '%' for d in descuentos]}")
    
    # 3. Especificaciones técnicas
    print("\n🔧 3. Especificaciones Técnicas:")
    
    # RAM
    patron_ram = r'([0-9]+GB)\s*(?:LPDDR[0-9]|DDR[0-9]|RAM)?'
    ram = re.findall(patron_ram, texto_mixto, re.IGNORECASE)
    extracciones['ram'] = list(set(ram))  # Eliminar duplicados
    print(f"  🧠 RAM encontrada: {extracciones['ram']}")
    
    # Almacenamiento
    patron_storage = r'([0-9]+(?:GB|TB))\s*(?:SSD|HDD|NVMe)?'
    storage = re.findall(patron_storage, texto_mixto, re.IGNORECASE)
    extracciones['almacenamiento'] = list(set(storage))
    print(f"  💾 Almacenamiento: {extracciones['almacenamiento']}")
    
    # Tamaño de pantalla
    patron_pantalla = r'([0-9]+(?:\.[0-9]+)?)\s*pulgadas'
    pantallas = re.findall(patron_pantalla, texto_mixto, re.IGNORECASE)
    extracciones['pantallas'] = [f'{p}"' for p in pantallas]
    print(f"  📺 Pantallas: {extracciones['pantallas']}")
    
    # Peso
    patron_peso = r'([0-9]+(?:\.[0-9]+)?)\s*kg'
    pesos = re.findall(patron_peso, texto_mixto, re.IGNORECASE)
    extracciones['pesos'] = [f'{p}kg' for p in pesos]
    print(f"  ⚖️ Pesos: {extracciones['pesos']}")
    
    # 4. Fechas
    print("\n📅 4. Fechas:")
    
    # Fechas en formato YYYY-MM-DD
    patron_fecha_iso = r'([0-9]{4}-[0-9]{2}-[0-9]{2})'
    fechas_iso = re.findall(patron_fecha_iso, texto_mixto)
    extracciones['fechas_iso'] = fechas_iso
    print(f"  📆 Fechas ISO: {fechas_iso}")
    
    # Fechas en formato DD/MM/YYYY
    patron_fecha_eu = r'([0-9]{1,2}/[0-9]{1,2}/[0-9]{4})'
    fechas_eu = re.findall(patron_fecha_eu, texto_mixto)
    extracciones['fechas_eu'] = fechas_eu
    print(f"  📆 Fechas EU: {fechas_eu}")
    
    # 5. URLs
    print("\n🔗 5. URLs:")
    
    # URLs completas
    patron_url = r'(https?://[A-Za-z0-9.-]+(?:/[A-Za-z0-9._~:/?#[\]@!$&\'()*+,;=-]*)?|ftp://[A-Za-z0-9.-]+(?:/[A-Za-z0-9._~:/?#[\]@!$&\'()*+,;=-]*)?)'
    urls = re.findall(patron_url, texto_mixto)
    extracciones['urls'] = urls
    print(f"  🌐 URLs encontradas: {len(urls)}")
    for i, url in enumerate(urls, 1):
        print(f"    {i}. {url}")
    
    # 6. Códigos y referencias
    print("\n🔢 6. Códigos y Referencias:")
    
    # SKUs (formato: XXX-XX-XX-XXX-XXX)
    patron_sku = r'([A-Z0-9]{3,}-[A-Z0-9]{2,}-[A-Z0-9]{2,}-[A-Z0-9]{3,}-[A-Z0-9]{3,})'
    skus = re.findall(patron_sku, texto_mixto)
    extracciones['skus'] = skus
    print(f"  📦 SKUs: {skus}")
    
    # Códigos de barras (12-13 dígitos)
    patron_barcode = r'\b([0-9]{12,13})\b'
    barcodes = re.findall(patron_barcode, texto_mixto)
    extracciones['barcodes'] = barcodes
    print(f"  📊 Códigos de barras: {barcodes}")
    
    # Seriales (formato: CXXXXXXXXXX)
    patron_serial = r'\b(C[0-9A-Z]{10,})\b'
    seriales = re.findall(patron_serial, texto_mixto)
    extracciones['seriales'] = seriales
    print(f"  🔑 Seriales: {seriales}")
    
    # 7. Stock y cantidades
    print("\n📦 7. Stock y Cantidades:")
    
    # Stock en formato "N unidades"
    patron_stock = r'stock:\s*([0-9]+)\s*unidades'
    stock = re.findall(patron_stock, texto_mixto, re.IGNORECASE)
    extracciones['stock'] = [int(s) for s in stock]
    print(f"  📦 Stock: {extracciones['stock']} unidades")
    
    return extracciones

# Ejecutar ejercicio
resultados_regex = ejercicio_regex_avanzado()

print("\n📊 RESUMEN DE EXTRACCIÓN REGEX:")
print("="*35)
total_elementos = 0
for categoria, datos in resultados_regex.items():
    cantidad = len(datos) if isinstance(datos, list) else 1
    total_elementos += cantidad
    print(f"📋 {categoria}: {cantidad} elementos")

print(f"\n🎯 Total de elementos extraídos: {total_elementos}")
print("✅ Ejercicio completado exitosamente")

## 💾 Creando un Sistema de Extracción Unificado

Finalmente, combinemos todo en un sistema robusto:

In [ ]:
# 💾 Sistema de extracción unificado

class ExtractorUnificado:
    """Sistema que combina XPath, CSS, Regex y BeautifulSoup"""
    
    def __init__(self, html_content):
        self.html_content = html_content
        self.tree = html.fromstring(html_content)
        self.soup = BeautifulSoup(html_content, 'html.parser')
        self.texto_completo = self.soup.get_text()
        
        # Compilar regex comunes para mejor performance
        self.regex_patterns = {
            'precios': re.compile(r'\$([0-9]{1,3}(?:,[0-9]{3})*(?:\.[0-9]{2})?)'),
            'emails': re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'),
            'telefonos': re.compile(r'(?:\+?1[-\s]?)?\(?([0-9]{3})\)?[-\s\.]?([0-9]{3})[-\s\.]?([0-9]{4})'),
            'urls': re.compile(r'https?://[A-Za-z0-9.-]+(?:/[A-Za-z0-9._~:/?#[\]@!$&\'()*+,;=-]*)?'),
        }
    
    def extraer_con_xpath(self, xpath_expr, tipo_retorno='text'):
        """Extraer datos usando XPath"""
        try:
            elementos = self.tree.xpath(xpath_expr)
            
            if tipo_retorno == 'text':
                return [elem.text.strip() if hasattr(elem, 'text') and elem.text else str(elem) for elem in elementos]
            elif tipo_retorno == 'attribute':
                return elementos  # Retorna los elementos para acceso a atributos
            else:
                return elementos
                
        except Exception as e:
            print(f"❌ Error en XPath '{xpath_expr}': {e}")
            return []
    
    def extraer_con_css(self, css_selector, atributo=None):
        """Extraer datos usando CSS selectors"""
        try:
            elementos = self.soup.select(css_selector)
            
            if atributo:
                return [elem.get(atributo, '') for elem in elementos]
            else:
                return [elem.get_text().strip() for elem in elementos]
                
        except Exception as e:
            print(f"❌ Error en CSS '{css_selector}': {e}")
            return []
    
    def extraer_con_regex(self, patron_nombre, texto_personalizado=None):
        """Extraer datos usando expresiones regulares"""
        try:
            texto_buscar = texto_personalizado or self.texto_completo
            
            if patron_nombre in self.regex_patterns:
                matches = self.regex_patterns[patron_nombre].findall(texto_buscar)
                return matches
            else:
                print(f"⚠️ Patrón '{patron_nombre}' no encontrado")
                return []
                
        except Exception as e:
            print(f"❌ Error en Regex '{patron_nombre}': {e}")
            return []
    
    def extraer_producto_completo(self, producto_id):
        """Extraer información completa de un producto usando todas las técnicas"""
        
        producto_data = {'id': producto_id}
        
        # 1. Datos básicos con XPath
        xpath_base = f'//article[@data-id="{producto_id}"]'
        
        # Título
        titulos = self.extraer_con_xpath(f'{xpath_base}//h3[@class="product-title"]')
        producto_data['titulo'] = titulos[0] if titulos else None
        
        # Marca
        marcas = self.extraer_con_xpath(f'{xpath_base}//span[@class="product-brand"]')
        producto_data['marca'] = marcas[0] if marcas else None
        
        # Atributos del elemento
        elemento_producto = self.tree.xpath(xpath_base)
        if elemento_producto:
            elem = elemento_producto[0]
            producto_data['precio_raw'] = elem.get('data-price')
            producto_data['stock'] = int(elem.get('data-stock', 0))
        
        # 2. Precios con CSS selectors
        css_base = f'article[data-id="{producto_id}"]'
        
        precio_actual = self.extraer_con_css(f'{css_base} .current-price')
        producto_data['precio_display'] = precio_actual[0] if precio_actual else None
        
        precio_original = self.extraer_con_css(f'{css_base} .original-price')
        producto_data['precio_original'] = precio_original[0] if precio_original else None
        
        descuento = self.extraer_con_css(f'{css_base} .discount-badge')
        producto_data['descuento_badge'] = descuento[0] if descuento else None
        
        # 3. Usar regex para limpiar precios
        if producto_data['precio_display']:
            precios_regex = self.extraer_con_regex('precios', producto_data['precio_display'])
            if precios_regex:
                producto_data['precio_numerico'] = float(precios_regex[0].replace(',', ''))
        
        # 4. Especificaciones con XPath + regex
        specs_elementos = self.extraer_con_xpath(f'{xpath_base}//div[@class="product-specs"]//li', 'attribute')
        especificaciones = {}
        
        for spec_elem in specs_elementos:
            texto_spec = spec_elem.text or ''
            
            # Procesador
            if 'Procesador:' in texto_spec:
                match = re.search(r'Procesador:\s*(.+)', texto_spec)
                if match:
                    especificaciones['procesador'] = match.group(1).strip()
            
            # RAM
            elif 'RAM:' in texto_spec:
                match = re.search(r'RAM:\s*([0-9]+GB)', texto_spec)
                if match:
                    especificaciones['ram'] = match.group(1)
        
        producto_data['especificaciones'] = especificaciones
        
        # 5. Rating con CSS
        rating = self.extraer_con_css(f'{css_base} .rating-value')
        if rating:
            try:
                producto_data['rating'] = float(rating[0])
            except ValueError:
                producto_data['rating'] = None
        
        # 6. Reviews count con regex
        review_elem = self.extraer_con_css(f'{css_base} .review-count')
        if review_elem:
            review_match = re.search(r'\((\d+)', review_elem[0])
            if review_match:
                producto_data['num_reviews'] = int(review_match.group(1))
        
        # 7. Badges con XPath
        badges = self.extraer_con_xpath(f'{xpath_base}//span[@class="badge"]')
        producto_data['badges'] = [b for b in badges if b.strip()]
        
        return producto_data
    
    def extraer_todos_los_productos(self):
        """Extraer información completa de todos los productos"""
        
        # Obtener IDs de productos
        ids_productos = self.extraer_con_css('article.product', 'data-id')
        
        productos = []
        for producto_id in ids_productos:
            if producto_id:
                producto = self.extraer_producto_completo(producto_id)
                productos.append(producto)
        
        return productos
    
    def generar_reporte_completo(self):
        """Generar un reporte completo del sitio"""
        
        print("📊 GENERANDO REPORTE COMPLETO")
        print("="*35)
        
        reporte = {
            'metadata': {
                'timestamp': datetime.now().isoformat(),
                'metodos_usados': ['XPath', 'CSS Selectors', 'Regex', 'BeautifulSoup']
            }
        }
        
        # Información general del sitio
        titulo_sitio = self.extraer_con_xpath('//h1[@class="site-title"]')
        reporte['sitio'] = {
            'nombre': titulo_sitio[0] if titulo_sitio else 'N/A',
            'total_elementos': len(self.tree.xpath('//*')),
            'total_enlaces': len(self.extraer_con_css('a'))
        }
        
        # Productos
        productos = self.extraer_todos_los_productos()
        reporte['productos'] = productos
        
        # Estadísticas
        productos_disponibles = [p for p in productos if p['stock'] > 0]
        precios_disponibles = [p['precio_numerico'] for p in productos_disponibles if p.get('precio_numerico')]
        
        reporte['estadisticas'] = {
            'total_productos': len(productos),
            'productos_disponibles': len(productos_disponibles),
            'productos_sin_stock': len(productos) - len(productos_disponibles),
            'precio_promedio': sum(precios_disponibles) / len(precios_disponibles) if precios_disponibles else 0,
            'precio_min': min(precios_disponibles) if precios_disponibles else 0,
            'precio_max': max(precios_disponibles) if precios_disponibles else 0
        }
        
        # Información de contacto extraída con regex
        emails = self.extraer_con_regex('emails')
        telefonos = self.extraer_con_regex('telefonos')
        urls = self.extraer_con_regex('urls')
        
        reporte['contacto'] = {
            'emails': emails,
            'telefonos': [f"({t[0]}) {t[1]}-{t[2]}" for t in telefonos],
            'urls': urls
        }
        
        return reporte

# Crear y usar el extractor unificado
print("🚀 CREANDO EXTRACTOR UNIFICADO")
print("="*35)

extractor = ExtractorUnificado(html_xpath_ejemplo)
reporte_final = extractor.generar_reporte_completo()

# Mostrar resultados
print(f"\n📋 REPORTE FINAL:")
print(f"🏪 Sitio: {reporte_final['sitio']['nombre']}")
print(f"📦 Productos totales: {reporte_final['estadisticas']['total_productos']}")
print(f"✅ Productos disponibles: {reporte_final['estadisticas']['productos_disponibles']}")
print(f"❌ Sin stock: {reporte_final['estadisticas']['productos_sin_stock']}")
print(f"💰 Precio promedio: ${reporte_final['estadisticas']['precio_promedio']:.2f}")
print(f"📧 Emails encontrados: {len(reporte_final['contacto']['emails'])}")
print(f"📞 Teléfonos encontrados: {len(reporte_final['contacto']['telefonos'])}")

# Detalle de productos
print(f"\n📦 DETALLE DE PRODUCTOS:")
for i, producto in enumerate(reporte_final['productos'], 1):
    disponibilidad = "✅ Disponible" if producto['stock'] > 0 else "❌ Sin stock"
    precio = f"${producto.get('precio_numerico', 'N/A'):.2f}" if producto.get('precio_numerico') else 'N/A'
    rating = f"⭐{producto.get('rating', 'N/A')}" if producto.get('rating') else '⭐N/A'
    
    print(f"  {i}. {producto.get('titulo', 'Sin título')} ({producto.get('marca', 'Sin marca')})")
    print(f"     💰 {precio} | 📦 Stock: {producto['stock']} | {rating} | {disponibilidad}")

print(f"\n✅ Sistema de extracción unificado completado exitosamente")

## 🎓 Resumen de la Lección

### 📚 Lo que hemos dominado:

1. **🎯 XPath completo**:
   - Sintaxis básica y avanzada
   - Predicados y funciones
   - Navegación por ejes (parent, sibling, ancestor)
   - Lógica compleja con operadores

2. **🔤 Expresiones regulares**:
   - Patrones para extraer precios, emails, teléfonos
   - Grupos nombrados y no nombrados
   - Modificadores de búsqueda (IGNORECASE, etc.)
   - Validación y limpieza de datos

3. **🔗 Combinación de técnicas**:
   - XPath + CSS Selectors + Regex
   - Estrategias para diferentes tipos de datos
   - Fallbacks y manejo de errores

4. **🤖 Introducción a contenido dinámico**:
   - Conceptos de Selenium
   - Cuándo usar qué herramienta
   - Esperas y timing

### 🛠️ Herramientas y métodos clave:

#### XPath esenciales:
```xpath
// = buscar en cualquier lugar
[@attribute="value"] = filtrar por atributo
[text()="texto"] = filtrar por contenido
[contains(text(), "parte")] = contiene texto
[position()=1] = primer elemento
[last()] = último elemento
//parent::* = elemento padre
//following-sibling::* = hermanos siguientes
```

#### Regex útiles:
```python
# Precios
r'\$([0-9]{1,3}(?:,[0-9]{3})*(?:\.[0-9]{2})?)'

# Emails
r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Teléfonos US
r'(?:\+?1[-\s]?)?\(?([0-9]{3})\)?[-\s\.]?([0-9]{3})[-\s\.]?([0-9]{4})'
```

### 🎯 Cuándo usar cada técnica:

| Técnica | Mejor para | Ejemplo |
|---------|------------|----------|
| **XPath** | Navegación compleja, predicados | `//div[@class="price"][../span[text()="Sale"]]` |
| **CSS** | Selección rápida y simple | `.product .price` |
| **Regex** | Extraer patrones de texto | Precios, emails, fechas |
| **BeautifulSoup** | Navegación DOM en Python | `.parent`, `.find_next_sibling()` |

### 🚀 Mejores prácticas:

1. **🎯 Combina técnicas** - Usa la herramienta correcta para cada tarea
2. **⚡ Optimiza performance** - Compila regex, usa selectores eficientes
3. **🛡️ Maneja errores** - Siempre ten fallbacks
4. **📊 Valida datos** - Verifica que los datos extraídos sean coherentes
5. **🧪 Prueba exhaustivamente** - Usa diferentes ejemplos de HTML

### 🔮 Próximos pasos:

En la siguiente lección aprenderemos:
- **CSS Selectors avanzados** y pseudoclases
- **Chaining methods** para extracciones complejas
- **Optimización de selectores** para mejor performance
- **Debugging y troubleshooting** de selectores

---

¡Increíble progreso! 🎉 Ahora tienes un arsenal completo de técnicas de selección. Puedes extraer prácticamente cualquier dato de cualquier página web. En la próxima lección perfeccionaremos estas habilidades y aprenderemos técnicas de encadenamiento avanzadas.